In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from pprint import pprint
from functools import lru_cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase

In [50]:
DAY = 7

In [51]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'$ cd /\n$ ls\ndir bqc\ndir mwmlf\ndir ngn\n143562 nrwjb\n78449 qqvdcclf\ndir qrnm\ndir s'
Split Data:
['$ cd /',
 '$ ls',
 'dir bqc',
 'dir mwmlf',
 'dir ngn',
 '143562 nrwjb',
 '78449 qqvdcclf',
 'dir qrnm',
 'dir smfzmmhc',
 '116085 tvrms']


In [52]:
file_re = re.compile(r"^(\d+)\s+(.*)$")

In [53]:
class Directory:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.subdirs = []
        self.files = []
        self.size = 0
    
    def add_file(self, name, size):
        f = File(name, size, self)
        self.files.append(f)
    
    def add_subdir(self, name):
        if self.subdir_exists(name) is None:
            d = Directory(name, self)
            self.subdirs.append(d)
        
    def subdir_exists(self, name):
        for d in self.subdirs:
            if d.name == name:
                return d
        else:
            return None
        
    def calculate_size(self):
        size = sum([sub_d.calculate_size() for sub_d in self.subdirs]) + sum([f.size for f in self.files])
        self.size = size
        return size
    
    def iterate_over_size(self, limit=100_000, found=set()):
        if limit is None or self.size < limit:
            found.add((self.full_path(), self.size))
        for d in self.subdirs:
            found.update(d.iterate_over_size(limit, found))
        
        return found
    
    def print_dirs(self, indent=""):
        print(f"{indent}- {self.name} (dir, size={self.size})")
        for d in self.subdirs:
            d.print_dirs(indent + " "*2)
        for f in self.files:
            print(indent + " "*2 + str(f))
    
    def full_path(self):
        path = (self.name,)
        parent = self.parent
        while True:
            if parent is None:
                return path
            else:
                path = (parent.name,) + path
                parent = parent.parent
        

In [54]:
class File:
    def __init__(self, name, size, parent=None):
        self.name = name
        self.size = int(size)
        self.parent = parent
    
    def __str__(self):
        return f"- {self.name} (file, size={self.size})"

In [55]:
def process_instructions(data):
    top_level = Directory("/")
    current_directory: Directory = top_level
    i = 0
    while True:
        try:
            line = data[i]
        except IndexError:
            break
        if line.startswith("$"):
            if line.startswith("$ cd"):
                if line.startswith("$ cd /"):
                    pass
                elif line.startswith("$ cd .."):
                    current_directory = current_directory.parent
                else:
                    #move into directory
                    name = line[5:]
                    current_directory = current_directory.subdir_exists(name)
                    pass
            if line.startswith("$ ls"):
                while True:
                    i+=1
                    try:
                        ls_line = data[i]
                    except IndexError:
                        break
                    if ls_line.startswith("dir "):
                        name = ls_line[4:]
                        current_directory.add_subdir(name)
                    elif ls_line.startswith("$"):
                        i-=1
                        break
                    else:
                        # file
                        size, file_name = file_re.match(ls_line).groups()
                        current_directory.add_file(file_name, size)
                    
        i+=1
    return top_level

In [56]:
overall = process_instructions(data)
overall.calculate_size()
# overall.print_dirs()
sizes = overall.iterate_over_size()

In [57]:
first = sum([s for path,s in sizes])

In [58]:
submit_answer(DAY, 1, first)

1325919
("You don't seem to be solving the right level.  Did you already complete it? "
 '[Return to Day 7]')


False

In [59]:
TOTAL = 70000000
NEED = 30000000

In [60]:
sizes = overall.iterate_over_size(limit=None)
used = overall.size
now_free = TOTAL-used
must_free = NEED - now_free
print(TOTAL, NEED, used, now_free, must_free)

for path, s in sorted(sizes, key=lambda x: x[1]):
    if s >= must_free:
        print(path, s)
        second = s
        break

70000000 30000000 42036703 27963297 2036703
('/', 'smfzmmhc', 'vwfnglr', 'mtvgvvcr') 2050735


In [61]:
submit_answer(DAY, 2, second)

2050735
("You don't seem to be solving the right level.  Did you already complete it? "
 '[Return to Day 7]')


False